<a href="https://colab.research.google.com/github/OlajideFemi/Carbon-Footprint/blob/main/Mixture_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Inputs: V (N x D with values {0,1} and np.nan for missing), H=2
# Outputs: pi (H,), theta (H x D), responsibilities R (N x H)

init pi = np.ones(H)/H
init theta = rng.uniform(0.25, 0.75, size=(H, D))

for iter in range(max_iters):
    # E-step
    logR = np.log(pi)[None, :]  # (1 x H) broadcast to (N x H)
    for k in range(H):
        # add sum over observed dims: v*log(theta) + (1-v)*log(1-theta)
        term = 0
        for d in range(D):
            obs = ~np.isnan(V[:, d])
            v = V[obs, d]
            loglik = v*np.log(theta[k, d]) + (1 - v)*np.log(1 - theta[k, d])
            logR[obs, k] += loglik
    # normalize
    R = softmax(logR, axis=1)

    # M-step
    Nk = R.sum(axis=0)              # (H,)
    pi = Nk / N
    for k in range(H):
        for d in range(D):
            obs = ~np.isnan(V[:, d])
            num = (R[obs, k] * V[obs, d]).sum()
            den = (R[obs, k]).sum()
            if den > 0:
                theta[k, d] = num / den
            # else: keep theta[k,d] or apply prior


SyntaxError: invalid syntax (ipython-input-3667588727.py, line 4)

In [1]:
import numpy as np
import pandas as pd

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
num_respondents = 150
num_questions = 5
missing_percentage = 0.30

# Generate a base dataset with 'yes' (1) and 'no' (0) responses
# We use a random binomial distribution for this example
data = np.random.randint(2, size=(num_respondents, num_questions))
df = pd.DataFrame(data, columns=[f'Q{i+1}' for i in range(num_questions)])

# Convert numerical responses to 'Yes' and 'No' for clarity
df = df.replace({1: 'Yes', 0: 'No'})

# Introduce missing data
missing_mask = np.random.rand(num_respondents, num_questions) < missing_percentage
df_missing = df.mask(missing_mask, pd.NA)

# Add a 'Respondent_ID' column
df_missing.insert(0, 'Respondent_ID', range(1, num_respondents + 1))

# Save the DataFrame to a CSV file
output_filename = 'questionnaire_data.csv'
df_missing.to_csv(output_filename, index=False)

print(f"Dataset generated and saved as '{output_filename}'")
print("\nFirst 5 rows of the generated dataset:")
print(df_missing.head())
print(f"\nTotal missing values: {df_missing.isnull().sum().sum()}")
print(f"Total possible values: {num_respondents * num_questions}")

Dataset generated and saved as 'questionnaire_data.csv'

First 5 rows of the generated dataset:
   Respondent_ID    Q1    Q2    Q3   Q4    Q5
0              1    No   Yes  <NA>   No  <NA>
1              2  <NA>    No    No   No   Yes
2              3    No  <NA>  <NA>   No   Yes
3              4    No   Yes   Yes  Yes    No
4              5   Yes    No   Yes  Yes   Yes

Total missing values: 237
Total possible values: 750


In [2]:
df_missing.shape

(150, 6)

In [4]:
# file_path = "/mnt/data/questionnaire_data.csv"
# df = pd.read_csv(file_path)
df = df_missing

In [5]:
# Keep Respondent_ID separate
respondent_ids = df["Respondent_ID"].values

In [6]:
# Replace Yes/No with 1/0, keep NaN as missing
df_clean = df.drop(columns=["Respondent_ID"]).replace({"Yes": 1, "No": 0})


In [7]:
df_clean

,Q1,Q2,Q3,Q4,Q5
0,0,1,<NA>,0,<NA>
1,<NA>,0,0,0,1
2,0,<NA>,<NA>,0,1
3,0,1,1,1,0
4,1,0,1,1,1
...,...,...,...,...,...
145,<NA>,0,0,0,1
146,<NA>,0,0,1,<NA>
147,0,<NA>,<NA>,0,1
148,0,<NA>,1,<NA>,<NA>
